# Imports

In [2]:
from di_python.scalebar import scale_bar 

In [98]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from netCDF4 import Dataset as netcdf_dataset
import netCDF4
import cartopy.geodesic as cge
import wrf
import cartopy

In [3]:
import importlib
import di_python
import di_python.ya_esta as ye

In [4]:
importlib.reload(ye)
ye.LOLA_CHC

[-68.131389, -16.353333]

# Constants 

In [9]:
LANDUSE_DIC = {
1:'Evergreen Needleleaf Forest',
2:'Evergreen Broadleaf Forest',
3:'Deciduous Needleleaf Forest',
4:'Deciduous Broadleaf Forest',
5:'Mixed Forests',
6:'Closed Shrublands',
7:'Open Shrublands',
8:'Woody Savannas',
9:'Savannas',
10:'Grasslands',
11:'Permanent Wetlands',
12:'Croplands',
13:'Urban and Built-Up',
14:'Cropland/Natural Vegetation Mosaic',
15:'Snow and Ice',
16:'Barren or Sparsely Vegetated',
17:'Water',
18:'Wooded Tundra',
19:'Mixed Tundra',
20:'Barren Tundra',
21:'Unclassified',
}

In [31]:
REF_LAT = -17
REF_LON = -66.3
REF_LOLA = [REF_LON, REF_LAT]

# Functions  

In [40]:
def get_lola(path):
    file_ds = netcdf_dataset(path, 'r')
    latds = wrf.getvar(file_ds,"CLAT")
    la,lo=wrf.latlon_coords(latds)
    la0=la.values[0,0]
    la1=la.values[-1,-1]
    lo0=lo.values[0,0]
    lo1=lo.values[-1,-1]
    return lo0,lo1,la0,la1




In [44]:
def ax_draw_sq(ax,lola):
    lo0,lo1,la0,la1 = lola
   
    ax.plot(
        [lo0,lo0,lo1,lo1,lo0],
        [la0,la1,la1,la0,la0],
        transform=cartopy.crs.PlateCarree(),
        c='black'
        )
    
    return ax

In [35]:
def lola_plus_dis(lola,ang,dis_m):
    geo = cge.Geodesic()
    res = geo.direct(lola, ang, dis_m).base
    return res

In [5]:
def get_file_list(dir_path, ext='wrfout_d0'):
    all_files = pd.DataFrame(os.listdir(dir_path),columns = ['name'])
    boo = all_files.name.str.contains(ext) & all_files.name.str.contains('d0')
    met_em_files = all_files[boo]
    met_em_files = met_em_files.sort_values('name')
    met_em_files['path']=met_em_files.apply(
        lambda x: 
            os.path.join(dir_path,x['name']),
        axis=1)
    met_em_files = met_em_files.reset_index(drop=True)
    met_em_files['dom'] = met_em_files.name.str.extract(r'(d0\d*)')
    return met_em_files

# Playground 